### Imports

In [2]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
import trainer_lib as tl

torch.manual_seed(2412231200)
random.seed(3009231410)
np.random.seed(2909231846)
np_random_state = np.random.RandomState(131002)

### Load data

In [3]:
df: pd.DataFrame = tl.load_country_wide_dataset('../data/country_data.csv')

X = df['el_load'].to_numpy(dtype=np.float32)
y = X.copy()

### Define models

In [18]:
class ConvNetSmall(nn.Module):
    def __init__(self, dropout=0.5, **kwargs):
        super(ConvNetSmall, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(1, 8, 5, padding=2),
            nn.BatchNorm1d(8),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(8, 16, 5, padding=2),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(2),
        )
        self.fc = nn.Sequential(
            nn.Flatten(1, -1),
            nn.Dropout(dropout),
            nn.Linear(6 * 16, 3),
        )

    def forward(self, x):
        x = x.reshape(-1, 1, 24)
        x = self.conv(x)
        x = self.fc(x)
        return x
    
class ConvNetMedium(nn.Module):
    def __init__(self, dropout=0.5, **kwargs):
        super(ConvNetMedium, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(1, 16, 7, padding=3),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Dropout(dropout / 5),
            nn.Conv1d(16, 32, 7, padding=3),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Dropout(dropout / 5),
            nn.Conv1d(32, 32, 7, padding=3),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(2),
        )
        self.fc = nn.Sequential(
            nn.Flatten(1, -1),
            nn.Dropout(dropout),
            nn.Linear(3 * 32, 3),
        )

    def forward(self, x):
        x = x.reshape(-1, 1, 24)
        x = self.conv(x)
        x = self.fc(x)
        return x
    
class ConvNetLarge(nn.Module):
    def __init__(self, dropout=0.5, **kwargs):
        super(ConvNetLarge, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(1, 32, 9, padding=4),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Dropout(dropout / 5),
            nn.Conv1d(32, 64, 9, padding=4),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Dropout(dropout / 5),
            nn.Conv1d(64, 64, 9, padding=4),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(2),
        )
        self.fc = nn.Sequential(
            nn.Flatten(1, -1),
            nn.Dropout(dropout / 2),
            nn.Linear(3 * 64, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 3),
        )

    def forward(self, x):
        x = x.reshape(-1, 1, 24)
        x = self.conv(x)
        x = self.fc(x)
        return x

tensor([[-0.3949, -0.0125, -0.1370]], grad_fn=<AddmmBackward0>)

### Grid search

In [19]:
grid = tl.Grid({
    'epochs': [200],  # we use early stop anyway
    'batch_size': [64], # only 1 option, so we have less params to search, this can be a single test afterward
    'lr': [0.001, 0.0001],
    'model': [ConvNetSmall, ConvNetMedium, ConvNetLarge],
    'dropout': [0.5],
    'n_splits': [6],    
})

wrapper = tl.MIMOTSWrapper(ConvNetSmall(), seq_len=24, pred_len=3)
b_p, b_s = wrapper.grid_search(X, y, grid, verbose=4)
print(f"Best params: {b_p}\nBest score: {b_s}")

[Grid search 001] BEGIN - params: {'epochs': 200, 'batch_size': 64, 'lr': 0.001, 'model': <class '__main__.ConvNetSmall'>, 'dropout': 0.5, 'n_splits': 6}
[Fold 1] BEGIN
Early stopping... Epoch 026: train loss: 0.100978, val loss: 0.108640, test loss: 0.172985
[Fold 1] END - RMSE loss: 261.655 - Time: 0.3 min.
[Fold 2] BEGIN
Early stopping... Epoch 022: train loss: 0.098946, val loss: 0.098821, test loss: 0.052177
[Fold 2] END - RMSE loss: 151.252 - Time: 0.4 min.
[Fold 3] BEGIN
Early stopping... Epoch 039: train loss: 0.090542, val loss: 0.085512, test loss: 0.034996
[Fold 3] END - RMSE loss: 127.090 - Time: 0.9 min.
[Fold 4] BEGIN
Early stopping... Epoch 041: train loss: 0.090202, val loss: 0.150568, test loss: 0.087215
[Fold 4] END - RMSE loss: 194.021 - Time: 1.2 min.
[Fold 5] BEGIN
Early stopping... Epoch 032: train loss: 0.092739, val loss: 0.094023, test loss: 0.121490
[Fold 5] END - RMSE loss: 241.328 - Time: 1.2 min.
[Fold 6] BEGIN
Early stopping... Epoch 034: train loss: 0.098